In [1]:
from pathlib import Path

import pandas as pd
import torch
import torch.optim as optim

from helpers.evals import evaluate_model
from helpers.features import process_dataset
from helpers.loaders import prepare_data_for_pytorch
from helpers.loss import ScientificDerivedLoss
from helpers.models import FrequencyAwareNetwork
from helpers.trainers import train_model

### Config


In [2]:
ANALYSIS = False
VERBOSE = True

DATASET_FILE_PATH = "dataset.csv"

GRAPH_FOLDER = "graphs"
MODELS = "models"
PREDICTIONS = "predictions"
SUBFOLDER = "baseline"

### Data


In [3]:
df = pd.read_csv(DATASET_FILE_PATH)

(
    X_train,
    Y_train,
    X_test,
    Y_test,
    voltage_scaler,
    freq_scaler,
) = process_dataset(df, test_size=0.2, random_state=42)

### Training


In [4]:
best_hparams = {
    "hidden_sizes": [384, 768, 1536],
    "dropout_rate": 0.1,
    "learning_rate": 0.002,
    "batch_size": 2048,
    "epochs": 300,
    "patience": 40,
    "activation": "gelu",
    "lr_scheduler_type": "reduce_on_plateau",
}

model_dir = Path(MODELS) / SUBFOLDER
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
freq_idx = [X_train.columns.get_loc("freq")]
other_idx = [i for i in range(X_train.shape[1]) if i not in freq_idx]

results = {}

In [ ]:
# Define all 4 S-parameter output labels (real and imaginary parts)
s_labels = [
    "S_deemb(1,1)_real",
    "S_deemb(2,1)_real",
    "S_deemb(1,2)_real",
    "S_deemb(2,2)_real",
    "S_deemb(1,1)_imag",
    "S_deemb(2,1)_imag",
    "S_deemb(1,2)_imag",
    "S_deemb(2,2)_imag",
]

# Extract target values for those S-parameters
y_train_full = Y_train[s_labels]
y_test_full = Y_test[s_labels]

# Prepare training/test tensors and loader
(
    X_train_tensor,
    Y_train_tensor,
    X_test_tensor,
    Y_test_tensor,
    train_loader,
    _,
    freq_test_tensor,
) = prepare_data_for_pytorch(
    X_train,
    y_train_full,
    X_test,
    y_test_full,
    batch_size=best_hparams["batch_size"],
    scale_y=False,  # no scaling for physics-based outputs
)

# Build the model with 8-output structure
model = FrequencyAwareNetwork(
    freq_indices=freq_idx,
    other_indices=other_idx,
    hidden_sizes=best_hparams["hidden_sizes"],
    dropout_rate=best_hparams["dropout_rate"],
    activation=best_hparams["activation"],
)

# Use ScientificDerivedLoss (works on S → derived physics)
criterion = ScientificDerivedLoss()
optimizer = optim.Adam(model.parameters(), lr=best_hparams["learning_rate"])

# Train model
trained_model = train_model(
    model,
    train_loader,
    X_test_tensor,
    Y_test_tensor,
    criterion,
    optimizer,
    device,
    epochs=best_hparams["epochs"],
    patience=best_hparams["patience"],
    scheduler_str=best_hparams["lr_scheduler_type"],
    freq_test=freq_test_tensor,  # pass frequency for validation
)

# Evaluate accuracy directly on predicted S-parameters
metrics = evaluate_model(
    trained_model, X_test_tensor.to(device), Y_test_tensor.to(device)
)

Training Epochs:  15%|█▌        | 46/300 [00:36<03:21,  1.26it/s, Epoch=46, Val Loss=28283070464.000000, Best=28283070464.000000, LR=2e-9]  

Early stopping triggered.


### Results


In [ ]:
print("Performance metrics for each S-parameter component:")
for i, label in enumerate(s_labels):
    print(f"\n{label}:")
    print(f"  R²:   {metrics['R2'][i]:.4f}")
    print(f"  MAE:  {metrics['MAE'][i]:.4f}")
    print(f"  RMSE: {metrics['RMSE'][i]:.4f}")

Performance metrics for each S-parameter component:

S_deemb(1,1)_real:
  R²:   -2415.1226
  MAE:  26.4884
  RMSE: 29.4288

S_deemb(2,1)_real:
  R²:   -13.1371
  MAE:  12.2662
  RMSE: 13.8417

S_deemb(1,2)_real:
  R²:   -13067.1475
  MAE:  10.6119
  RMSE: 13.2735

S_deemb(2,2)_real:
  R²:   -382.3744
  MAE:  7.3480
  RMSE: 8.5558

S_deemb(1,1)_imag:
  R²:   -1517.6161
  MAE:  8.2414
  RMSE: 10.0743

S_deemb(2,1)_imag:
  R²:   -176.8819
  MAE:  24.8440
  RMSE: 29.4351

S_deemb(1,2)_imag:
  R²:   -28584.6074
  MAE:  11.8184
  RMSE: 14.1460

S_deemb(2,2)_imag:
  R²:   -2084.7585
  MAE:  6.9029
  RMSE: 8.0568
